# Loading

In [1]:
import sys, os, re, csv, codecs
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model

from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [2]:
path = '../input/'
EMBEDDING_FILE = path + 'glove6b100dtxt/glove.6B.100d.txt'
TRAIN_DATA_FILE = path + 'train.csv'
TEST_DATA_FILE = path + 'test.csv'

In [3]:
train_df = pd.read_csv(TRAIN_DATA_FILE)
test_df = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train_df["comment_text"].fillna("_na_").values

class_list = ["toxic", "severe_toxic", "obscene", 
              "threat", "insult", "identity_hate"]
labels = train_df[class_list].values

list_sentences_test = test_df["comment_text"].fillna("_na_").values

# Preprocessing

In [4]:
### basic config param
embed_size = 100
max_features = 20000
maxlen = 100

### Generate sentence feature

In [5]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [6]:
features_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
features_test = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [7]:
features_train.shape

(159571, 100)

### Read in Glove

In [8]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

In [9]:
embeddings_index = dict(
    get_coefs(*o.strip().split()
             ) for o in open(EMBEDDING_FILE, encoding="utf-8"))

Create embedding matrix, with random initialization for words not in glove

In [10]:
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()

In [11]:
# generate random number matrix as place holder
word_index = tokenizer.word_index
nb_words = max(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, 
                                    (nb_words, embed_size))

In [12]:
# insert glove word vectors into the embedding matrix accoding to word index
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [13]:
embedding_matrix.shape

(210337, 100)

In [14]:
features_train[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,   52, 2635,   13,  555, 3809,
         73, 4556, 2706,   21,   94,   38,  803, 2679,  992,  589, 8377,
        182])

# Model

In [15]:
inp = Input(shape=(maxlen,))
x = Embedding(nb_words, embed_size, weights=[embedding_matrix])(inp)
#x = Embedding(nb_words, embed_size)(inp)
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              validation_split=0.01，
              metrics=['accuracy'])

# Training

keras KeyError: do NOT use dataframe as input, but use np array

In [16]:
model.fit(features_train, labels, batch_size=64, epochs=10)

Epoch 1/10
159571/159571 [==============================] - 954s - loss: 0.0582 - acc: 0.9797   
Epoch 2/10
159571/159571 [==============================] - 56075s - loss: 0.0429 - acc: 0.9836 
Epoch 3/10
159571/159571 [==============================] - 668s - loss: 0.0381 - acc: 0.9850   
Epoch 4/10
159571/159571 [==============================] - 660s - loss: 0.0345 - acc: 0.9861   
Epoch 5/10
159571/159571 [==============================] - 651s - loss: 0.0311 - acc: 0.9873   
Epoch 6/10
159571/159571 [==============================] - 665s - loss: 0.0281 - acc: 0.9884   
Epoch 7/10
159571/159571 [==============================] - 646s - loss: 0.0249 - acc: 0.9897   
Epoch 8/10
159571/159571 [==============================] - 664s - loss: 0.0220 - acc: 0.9910   
Epoch 9/10
159571/159571 [==============================] - 666s - loss: 0.0194 - acc: 0.9921   
Epoch 10/10
159571/159571 [==============================] - 2170s - loss: 0.0173 - acc: 0.9929  


# Generate submission

In [17]:
y_test = model.predict([features_test], batch_size=1024, verbose=1)
sample_submission = pd.read_csv(path+'sample_submission.csv')


sample_submission[class_list] = y_test
sample_submission.to_csv('lstm_glove0226_100.csv', index=False)

153164/153164 [==============================] - 20s    
